## Keras建立多层感知器模型（MLP），递归神经网络(RNN)，长短期记忆模型(LSTM)

## 14.1 建立多层感知器模型进行IMDb情感分析

### 建立多层感知器模型，如下图所示：
![](./lesson14/2.jpg)

- ### 嵌入层： 将“数字列表”转换为“向量列表”
- ### 多层感知器： 使用多层感知器模型处理“向量列表”
    - #### 平坦层： 共有3200个神经元，因为原本“数字列表”每一项有100个数字，每一个数字转换为32维的向量，所有转换成的平坦层的神经元有3200个(32*100=3200)
    - #### 隐藏层：共有256个神经元
    - #### 输出层：只有1个神经元，输出1代表正面评价，0代表负面评价

## 14.2 数据预处理

- ### 预处理的主要过程，在上章有介绍

In [10]:
# 1. 导入所需模块
from keras.datasets import imdb
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer

In [11]:
import os
# 2. 创建 rm_tag 函数，删除文字中的HTML标签
import re
def rm_tag(text):
    re_tag = re.compile(r'<[^>]+>')
    return re_tag.sub(' ',  text)

# 3. 创建read_files函数读取IMDb文件目录
def read_files(filetype):    # 创建read_files 函数， 读取训练数据时 filetype会传入 train， 读取测试数据时 filetype 会传入 test
    path = "./data/aclImdb/"    # 设置存放文件的路径
    file_list = []    # 创建文件列表
    
    postive_path = path+filetype+"/pos/"    # 设置正面评价的文件目录为 positive_path
    for f in os.listdir(postive_path):    # 使用for循环，将positive_path目录下的所有文件加入file_list
        file_list += [postive_path+f]
        
    negative_path = path+filetype+"/neg/"    # 设置负面评价的文件目录为negative_path
    for f in os.listdir(negative_path):    # 使用for循环，将positive_path目录下的所有文件加入file_list
        file_list += [negative_path+f]
        
    print('read ', filetype, 'files :', len(file_list))    # 显示当前读取的filetype 【'train', 'test'】下的文件个数
    
    all_labels = ([1]*12500 + [0]*12500)    # 本条语句产生all_labels, 因为 前12500项是正面，所以产生12500项1的列表。 后12500项是负面，产生12500项0的列表
    
    all_texts = []
    
    for fi in file_list:    # 读取file_list 的所有文件
        with open(fi, encoding='utf-8') as file_input:    # 使用 (fi, encoding='utf-8') 打开文件file_input
            all_texts += [rm_tag(" ".join(file_input.readlines()))]    # file_input.readlines()读取文件，并使用join 连接所有文件内容，然后使用rm_tag删除tag，最后加入all_tests list
    
    return all_labels, all_texts    # 返回label, texts

In [12]:
# 2.读取IMDb数据集
# 读取训练数据集
y_train, train_text = read_files("train")

read  train files : 25000


In [13]:
# 读取测试数据集
y_test, test_text  = read_files("test")

read  test files : 25000


In [5]:
# 3. 建立Token（字典）
token = Tokenizer(num_words=2000)

In [6]:
token.fit_on_texts(train_text)

In [7]:
# 4. 将“影评文字”转为“数字列表”
x_train_seq = token.texts_to_sequences(train_text)
x_test_seq = token.texts_to_sequences(test_text)

In [8]:
# 5. 截长补短，让所有“数字列表”长度为100
x_train = sequence.pad_sequences(x_train_seq, maxlen=100)
x_test =  sequence.pad_sequences(x_test_seq, maxlen=100)

![](./lesson14/3.jpg)

***

## 14.3 加入嵌入层

### Kears提供了嵌入层可以将“数字列表”转换为“向量列表”

In [9]:
# 1.导入所需模块
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers.embeddings import Embedding

#### 2. 建立模型  建立一个线性堆叠模型，后续只需要将各个神经网络加入模型即可

In [10]:
model = Sequential()

#### 3. 将嵌入层加入模型，把“数字列表”转换为“向量列表”

In [11]:
model.add(Embedding(output_dim=32,
                    input_dim=2000,
                   input_length=100))

# 加入Dropout层防止过拟合，每次迭代会随机的放弃20%的神经元
model.add(Dropout(0.2))

![](./lesson14/4.jpg)

## 14.4 建立多层感知器模型

### 嵌入层转换为“向量列表”后，就可以使用深度学习模型进行训练与预测了

#### 1 将平坦层加入模型

In [12]:
# 因为“数字列表”每一项有100个数字，每一个数字转换为32维的向量，所有平坦层的神经元有32×100=3200个
model.add(Flatten())

#### 2 将隐藏层加入模型

In [13]:
# 2 将隐藏层加入模型
model.add(Dense(units=256, activation='relu'))
model.add(Dropout(0.25))

- #### units=256: 隐藏层共有256个神经元
- #### activation='relu'：定义激活函数 ReLU

#### 3. 将输出层加入模型

In [14]:
# 3. 将输出层加入模型
model.add(Dense(units=1, activation='sigmoid'))

- ### units=1: 输出层只有一个神经元，输出1代表正面评价，0代表负面评价
- ### activation='sigmoid'：定义激活函数sigmoid

#### 以上模型已经基本搭建完成
***

#### 查看模型摘要

In [15]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 32)           64000     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 32)           0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 3200)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               819456    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
Total params: 883,713
Trainable params: 883,713
Non-trainable params: 0
_________________________________________________________________


## 14.5 训练模型

#### 定义训练方式

#### compile方法需要输入3个参数，loss, optimizer, metrics, 对这三个参数的介绍可以参考前面章节

In [16]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

#### 开始训练

#### model.fit 进行训练

In [17]:
train_history = model.fit(x_train, y_train, batch_size=100, epochs=20, verbose=2, validation_split=0.2)

Train on 20000 samples, validate on 5000 samples
Epoch 1/20
 - 8s - loss: 0.4747 - acc: 0.7595 - val_loss: 0.4431 - val_acc: 0.7974
Epoch 2/20
 - 7s - loss: 0.2676 - acc: 0.8913 - val_loss: 0.6439 - val_acc: 0.7196
Epoch 3/20
 - 7s - loss: 0.1534 - acc: 0.9440 - val_loss: 0.5982 - val_acc: 0.7726
Epoch 4/20
 - 7s - loss: 0.0793 - acc: 0.9722 - val_loss: 0.9852 - val_acc: 0.7166
Epoch 5/20
 - 8s - loss: 0.0444 - acc: 0.9851 - val_loss: 1.0860 - val_acc: 0.7414
Epoch 6/20
 - 8s - loss: 0.0370 - acc: 0.9868 - val_loss: 1.0029 - val_acc: 0.7694
Epoch 7/20
 - 8s - loss: 0.0316 - acc: 0.9887 - val_loss: 1.1113 - val_acc: 0.7654
Epoch 8/20
 - 8s - loss: 0.0287 - acc: 0.9898 - val_loss: 1.1079 - val_acc: 0.7754
Epoch 9/20
 - 8s - loss: 0.0263 - acc: 0.9900 - val_loss: 1.5183 - val_acc: 0.7190
Epoch 10/20
 - 8s - loss: 0.0236 - acc: 0.9912 - val_loss: 1.3400 - val_acc: 0.7550
Epoch 11/20
 - 7s - loss: 0.0246 - acc: 0.9910 - val_loss: 1.4404 - val_acc: 0.7420
Epoch 12/20
 - 7s - loss: 0.0199 - a

## 14.6 评估模型的准确率

In [18]:
scores = model.evaluate(x_test, y_test, verbose=1)
scores[1]

25000/25000 [==============================] - 2s 66us/step


0.80896

## 14.7 进行预测
### 前面建立了模型，并且完成了模型的训练，准确率达到还可以接受的0.8，接下来进行预测

In [19]:
predict = model.predict_classes(x_test, verbose=1)

25000/25000 [==============================] - 2s 64us/step


In [20]:
# 查看前10项预测结果
predict[:10]

array([[1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1]])

#### 从以上结果可知，前10项全部预测为正面评价

In [22]:
# 转换为1维数组
predict[:10].reshape(-1)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

## 14.8 查看测试数据预测结果

### 之前的预测结果是0和1，我们将创建display_test_Sentiment函数，显示正面评价或负面评价

In [31]:
Sentiment = {0:'负面的', 1:'正面的'}
def display_test_Sentiment(i):
    print(test_text[i])
    print('label真实值：',Sentiment[y_test[i]], '预测结果：', Sentiment[predict[i][0]])

In [32]:
# 显示第10项预测结果
display_test_Sentiment(10)

I loved this movie from beginning to end.I am a musician and i let drugs get in the way of my some of the things i used to love(skateboarding,drawing) but my friends were always there for me.Music was like my rehab,life support,and my drug.It changed my life.I can totally relate to this movie and i wish there was more i could say.This movie left me speechless to be honest.I just saw it on the Ifc channel.I usually hate having satellite but this was a perk of having satellite.The ifc channel shows some really great movies and without it I never would have found this movie.Im not a big fan of the international films because i find that a lot of the don't do a very good job on translating lines.I mean the obvious language barrier leaves you to just believe thats what they are saying but its not that big of a deal i guess.I almost never got to see this AMAZING movie.Good thing i stayed up for it instead of going to bed..well earlier than usual.lol.I hope you all enjoy the hell of this movi

#### 从以上执行结果可知，第10项数据原本是正面的，预测结果同样也是正面的

In [34]:
# 显示第12502项预测结果
display_test_Sentiment(12502)

First of all I hate those moronic rappers, who could'nt act if they had a gun pressed against their foreheads. All they do is curse and shoot each other and acting like cliché'e version of gangsters.  The movie doesn't take more than five minutes to explain what is going on before we're already at the warehouse There is not a single sympathetic character in this movie, except for the homeless guy, who is also the only one with half a brain.  Bill Paxton and William Sadler are both hill billies and Sadlers character is just as much a villain as the gangsters. I did'nt like him right from the start.  The movie is filled with pointless violence and Walter Hills specialty: people falling through windows with glass flying everywhere. There is pretty much no plot and it is a big problem when you root for no-one. Everybody dies, except from Paxton and the homeless guy and everybody get what they deserve.  The only two black people that can act is the homeless guy and the junkie but they're ac

#### 从以上执行结果可知，第12502项数据原本是负面的，预测结果同样也是负面的

## 14.9 查看《美女与野兽》的影评

### 1. 查找影评  http://www.imdb.com/title/tt2771200/reviews
### 2. 执行预测，将input_text变量送入之前建立的多层感知模型进行预测

In [36]:
input_text = 'How can this have so many talented actors and be this bad?'

In [37]:
# 执行数据预处理， 将“影评文字”转换为“数字列表”
input_seq = token.texts_to_sequences([input_text])

In [44]:
# 查看数字列表
print(input_seq)
print(len(input_seq[0]))

[[85, 66, 10, 24, 34, 107, 1015, 152, 2, 26, 10, 74]]
12


In [45]:
# 把“数字列表”长度转换为100
x_input = sequence.pad_sequences(input_seq, maxlen=100)

In [47]:
# 查看pad之后的input字符
print(x_input)
print(len(x_input[0]))

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0   85   66   10   24   34  107 1015  152    2   26
    10   74]]
100


In [48]:
# 使用多层感知器进行预测
input_predict = model.predict_classes(x_input)

In [49]:
# 查看预测结果
input_predict

array([[0]])

#### 预测结果是一个二维数组，里面只有一个元素。 可以看到预测为 负面评价。

### 再来一个

In [53]:
input_text2 = '''The movie from the first scene up to the end was emotionless and spiritless. I do appreciate that they didn't want to change the story much and keep the classic version for the most. Even when the movie was slightly tweaked, it was in the wrong dull way. Some tweaks don't really make sense and don't add up. I don't know whom should be blamed for the 'emotion delivery failure' is it the cast or the director? Emma Watson is good actress for some certain roles, but certainly not this one. She was very stiff. We couldn't differentiate her surprise from anger or love from sadness. In many scenes Belle and the Beast were talking so fast with no emotions, soul or spirit. It was as if they were rehearsing on a play by reading out loud from their scripts and the director was simply not there. Cinematography had a problem too. Too many close ups. The original cartoon had some close ups too, but Do they really have to copy everything as-is? The couple dancing scene cinematography could have been better. I really don't advise anyone to see this movie. Disney failed this time. It happens, though!. Hopefully we'll know what really happened.'''

In [59]:
input_text2_seq = token.texts_to_sequences([input_text2])

In [64]:
print(input_text2_seq)
print('length is :',len(input_text2_seq[0]))

[[1, 16, 35, 1, 82, 132, 52, 5, 1, 126, 12, 2, 9, 77, 1141, 11, 32, 157, 177, 5, 648, 1, 61, 72, 2, 397, 1, 352, 306, 14, 1, 87, 56, 50, 1, 16, 12, 1071, 8, 12, 7, 1, 351, 749, 92, 45, 88, 62, 93, 277, 2, 88, 759, 52, 9, 88, 120, 932, 140, 26, 14, 1, 6, 8, 1, 173, 38, 1, 163, 6, 48, 519, 14, 45, 809, 551, 17, 430, 20, 10, 27, 55, 12, 51, 71, 422, 37, 861, 35, 38, 115, 35, 7, 107, 135, 2, 1, 67, 658, 34, 699, 15, 53, 1432, 1351, 38, 1099, 8, 12, 13, 44, 32, 67, 19, 3, 293, 30, 882, 42, 1291, 35, 64, 2, 1, 163, 12, 327, 20, 46, 623, 65, 3, 435, 95, 95, 107, 487, 1969, 1, 200, 1068, 65, 45, 487, 1969, 95, 17, 77, 32, 62, 24, 5, 1035, 281, 13, 6, 1, 375, 1101, 132, 623, 96, 24, 73, 124, 9, 62, 88, 255, 5, 63, 10, 16, 907, 1192, 10, 54, 8, 567, 147, 120, 47, 62, 570]]
length is : 184


In [62]:
x_input2 = sequence.pad_sequences(input_text2_seq, maxlen=100)

In [63]:
print(x_input2)
print(len(x_input2[0]))

[[  71  422   37  861   35   38  115   35    7  107  135    2    1   67
   658   34  699   15   53 1432 1351   38 1099    8   12   13   44   32
    67   19    3  293   30  882   42 1291   35   64    2    1  163   12
   327   20   46  623   65    3  435   95   95  107  487 1969    1  200
  1068   65   45  487 1969   95   17   77   32   62   24    5 1035  281
    13    6    1  375 1101  132  623   96   24   73  124    9   62   88
   255    5   63   10   16  907 1192   10   54    8  567  147  120   47
    62  570]]
100


In [65]:
# 送入模型预测
input2_predict = model.predict_classes(x_input2)

In [68]:
input2_predict[0][0]

0

## 14.10 预测《美女与野兽》的影评是正面的还是负面的

### 把前面的命令整理证函数，以便于后面的预测

In [70]:
# predict_review 使用很简单， 只需要传入预测的字符串列表
def predict_review(input_text):
#     字符串整理成数字
    input_sequence = token.texts_to_sequences([input_text])
#     数字长度统一为100
    pad_input = sequence.pad_sequences(input_sequence, maxlen=100)
#     执行预测
    predict_class = model.predict_classes(pad_input)
#     输出结果
    print(Sentiment[predict_class[0][0]])

### 去IMDb网站筛选影评，执行预测

In [71]:
predict_review('''
Where do I start. This adaptation of Disney's 1991 Beauty and the Beast was an utter disappointment. Emma Watson as Belle was extremely unconvincing from the start to the end. She had the same expressions as the actress from Twilight. The animators did a terrible job with the Beast. He looked fake and lifeless. They could have used special makeup to create the beast similar to the Grinch where we get to see Jim Carrey's expressions. The side character animations were poorly executed. Overall I felt the film was rushed as there was lack of compassion and chemistry between the characters. There was a lot of CGI and green screen which could have been replaced by normal acting, because then why make an animated version of an animated film? This is by far the worst remake of an animated classic.
''')

负面的


![](./lesson14/5.jpg)

In [72]:
predict_review('''
I must say, Disney seems to be putting out live action movies of their successful cartoons just for the sake of putting out live action movies! I guess corporate thinking is the initial buzz will make it money on the first weekend! This is the worst of the live actions, the acting is boring, the story mundane, I do not care about the characters at all, it seems it's just made to show how cool Disney can use special effects! If you like the story, stick to the original cartoon!''')

负面的


![](./lesson14/6.jpg)

In [74]:
predict_review('''
A fabulous movie, I enjoyed every moment. So beautifully done that I would watch it again. It's a true musical as they used to be. I cried and laughed, it brought out many emotions. It's a great family film. The artistry and special effects make a great Disney style fantasy come to life. The music and songs were very pleasant in typical Disney fashion.''')

正面的


![](./lesson14/7.jpg)

***

## 14.11 文字处理时，使用较大字典提取更多文字

### 之前的模型预测准确率是0.80，我们希望能够在提高模型的准确率
- ### 提高字典的单词个数：原本2000， 增加为有3800个单词的字典
- ### 增加“数字列表”的长度：原本都是100个数字，现在改为380个数字

### 1. 数据预处理，修改如下

In [13]:
# 建立字典个数为3800个单词
token = Tokenizer(num_words=3800)
token.fit_on_texts(train_text)

In [79]:
print(token.index_word)

{1: 'the', 2: 'and', 3: 'a', 4: 'of', 5: 'to', 6: 'is', 7: 'in', 8: 'it', 9: 'i', 10: 'this', 11: 'that', 12: 'was', 13: 'as', 14: 'for', 15: 'with', 16: 'movie', 17: 'but', 18: 'film', 19: 'on', 20: 'not', 21: 'you', 22: 'are', 23: 'his', 24: 'have', 25: 'he', 26: 'be', 27: 'one', 28: 'all', 29: 'at', 30: 'by', 31: 'an', 32: 'they', 33: 'who', 34: 'so', 35: 'from', 36: 'like', 37: 'her', 38: 'or', 39: 'just', 40: 'about', 41: "it's", 42: 'out', 43: 'has', 44: 'if', 45: 'some', 46: 'there', 47: 'what', 48: 'good', 49: 'more', 50: 'when', 51: 'very', 52: 'up', 53: 'no', 54: 'time', 55: 'she', 56: 'even', 57: 'my', 58: 'would', 59: 'which', 60: 'only', 61: 'story', 62: 'really', 63: 'see', 64: 'their', 65: 'had', 66: 'can', 67: 'were', 68: 'me', 69: 'well', 70: 'than', 71: 'we', 72: 'much', 73: 'been', 74: 'bad', 75: 'get', 76: 'will', 77: 'do', 78: 'also', 79: 'into', 80: 'people', 81: 'other', 82: 'first', 83: 'great', 84: 'because', 85: 'how', 86: 'him', 87: 'most', 88: "don't", 89: '

In [80]:
# 将文字转换为数字序列

In [14]:
x_train_seq = token.texts_to_sequences(train_text)
x_test_seq = token.texts_to_sequences(test_text)

In [ ]:
# 截长补短，让所有数字序列的长度统一

In [15]:
x_train = sequence.pad_sequences(x_train_seq, maxlen=380)
x_test = sequence.pad_sequences(x_test_seq, maxlen=380)

In [16]:
# 查看长度
print(len(x_train_seq[0]))
print(len(x_train[0]))

116
380


### 2.建立模型， 修改如下

In [90]:
model = Sequential()
model.add(Embedding(output_dim=32,
                   input_dim=3800,
                   input_length=380))
model.add(Dropout(0.2))

- #### input_dim=3800: 输入纬度改为3800
- #### input_length=380: “数字长度”修改为380

In [91]:
model.add(Flatten())

In [92]:
model.add(Dense(units=256,
                activation='relu'))
model.add(Dropout(0.2))

In [93]:
model.add(Dense(units=1,
               activation='sigmoid'))

In [94]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 380, 32)           121600    
_________________________________________________________________
dropout_3 (Dropout)          (None, 380, 32)           0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 12160)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)               3113216   
_________________________________________________________________
dropout_4 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 257       
Total params: 3,235,073
Trainable params: 3,235,073
Non-trainable params: 0
_________________________________________________________________


In [95]:
# 修改“数字列表”长度为380
pad_input_seq = sequence.pad_sequences(input_seq, maxlen=380)

In [96]:
# 修改predict_review函数
def predict_review(input_text):
#     字符串转为数字
    input_seq = token.texts_to_sequences([input_text])
#     padding 字符串为统一长度
    pad_input_seq = sequence.pad_sequences(input_seq, maxlen=380)
#     执行预测
    predict_result = model.predict_classes(pad_input_seq)
#     打印预测结果
    print(Sentiment[predict_result[0][0]])

In [99]:
# 评估模型的准确率
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [101]:
train_history = model.fit(x_train, y_train, batch_size=100, epochs=10, verbose=2, validation_split=0.2)

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
 - 28s - loss: 0.4887 - acc: 0.7482 - val_loss: 0.3973 - val_acc: 0.8298
Epoch 2/10
 - 28s - loss: 0.2024 - acc: 0.9208 - val_loss: 0.5058 - val_acc: 0.7906
Epoch 3/10
 - 27s - loss: 0.0860 - acc: 0.9710 - val_loss: 0.6720 - val_acc: 0.7786
Epoch 4/10
 - 28s - loss: 0.0298 - acc: 0.9917 - val_loss: 0.7695 - val_acc: 0.7922
Epoch 5/10
 - 28s - loss: 0.0136 - acc: 0.9966 - val_loss: 0.5583 - val_acc: 0.8656
Epoch 6/10
 - 31s - loss: 0.0096 - acc: 0.9974 - val_loss: 0.8398 - val_acc: 0.8094
Epoch 7/10
 - 29s - loss: 0.0084 - acc: 0.9978 - val_loss: 0.9733 - val_acc: 0.7990
Epoch 8/10
 - 25s - loss: 0.0127 - acc: 0.9955 - val_loss: 1.1855 - val_acc: 0.7726
Epoch 9/10
 - 28s - loss: 0.0163 - acc: 0.9943 - val_loss: 1.0998 - val_acc: 0.7924
Epoch 10/10
 - 27s - loss: 0.0166 - acc: 0.9940 - val_loss: 1.1914 - val_acc: 0.7880


In [102]:
scores = model.evaluate(x=x_test, y=y_test, verbose=1)
scores[1]

25000/25000 [==============================] - 5s 219us/step


0.84412

### 经过修改模型后：把字典的单词数增加为3800，并且“数字列表”的长度增加为380。训练的时间较长，准确率也从0.8提高到了0.84

___

## 14.12 RNN模型（递归神经网络）

![](./lesson14/8.jpg)
![](./lesson14/9.jpg)

### 3. 以时间节点展开RNN模型 -> 将上面的图以时间节点展开，如下图所示

![](./lesson14/10.jpg)
![](./lesson14/11.jpg)

___

## 14.13 使用Keras RNN模型进行IMDb情感分析

### 整理流程如下图所示
![](./lesson14/12.jpg)

### 1 建立模型  使用代码SimpleRNN(units=16)建立16个神经元的RNN层

In [1]:
# 导入相关模块
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import SimpleRNN

In [2]:
# 建立线性堆叠模型
model = Sequential()

In [3]:
# 加入嵌入层，将“数字列表”转换为“向量列表”
model.add(Embedding(output_dim=32,
                    input_dim=3800,
                    input_length=380))
model.add(Dropout(0.35))

In [4]:
# 建立RNN层， 16个神经元
model.add(SimpleRNN(units=16))

In [5]:
model.add(Dense(units=256, activation='relu'))
model.add(Dropout(0.35))

In [6]:
model.add(Dense(units=1, activation='sigmoid'))

In [7]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 380, 32)           121600    
_________________________________________________________________
dropout_1 (Dropout)          (None, 380, 32)           0         
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 16)                784       
_________________________________________________________________
dense_1 (Dense)              (None, 256)               4352      
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
Total params: 126,993
Trainable params: 126,993
Non-trainable params: 0
_________________________________________________________________
None

In [18]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [21]:
model.fit(x_train, y_train, validation_split=0.2, verbose=2, batch_size=100, epochs=10)

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
 - 90s - loss: 0.5789 - acc: 0.6930 - val_loss: 0.3281 - val_acc: 0.8862
Epoch 2/10
 - 125s - loss: 0.3612 - acc: 0.8470 - val_loss: 0.5830 - val_acc: 0.7850
Epoch 3/10
 - 137s - loss: 0.3028 - acc: 0.8774 - val_loss: 0.5189 - val_acc: 0.7592
Epoch 4/10
 - 135s - loss: 0.2661 - acc: 0.8950 - val_loss: 0.6026 - val_acc: 0.6838
Epoch 5/10
 - 132s - loss: 0.2314 - acc: 0.9095 - val_loss: 0.5561 - val_acc: 0.7914
Epoch 6/10
 - 151s - loss: 0.1727 - acc: 0.9365 - val_loss: 0.4286 - val_acc: 0.8510
Epoch 7/10
 - 144s - loss: 0.1530 - acc: 0.9431 - val_loss: 0.5447 - val_acc: 0.8218
Epoch 8/10
 - 148s - loss: 0.1338 - acc: 0.9507 - val_loss: 0.5887 - val_acc: 0.8072
Epoch 9/10
 - 147s - loss: 0.1092 - acc: 0.9605 - val_loss: 0.7084 - val_acc: 0.8084
Epoch 10/10
 - 164s - loss: 0.0926 - acc: 0.9666 - val_loss: 1.0237 - val_acc: 0.7404


In [22]:
# 评估模型的准确率
scores = model.evaluate(x_test, y_test, verbose=1)

25000/25000 [==============================] - 201s 8ms/step


In [23]:
scores[1]

0.82824

### 使用RNN模型准确率大概为0.82

___

## 14.14 LSTM模型介绍
### LSTM（Long Short Term Memory）长短期记忆，也是一种时间递归神经网络，专门设计来解决RNN的长期依赖问题

![](./lesson14/13.jpg)
![](./lesson14/14.jpg)
### 2.LSTM 介绍
![](./lesson14/15.jpg)
![](./lesson14/16.jpg)

***

## 14.15 使用Keras LSTM模型进行IMDb情感分析

### 通过上节的介绍LSTM的基本工作机制，实现情感分析

In [1]:
# 导入相关依赖
from keras.models import Sequential
from keras.layers import Embedding
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.recurrent import LSTM

Using TensorFlow backend.
F:\python\Anaconda3\envs\tensorFLow-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
F:\python\Anaconda3\envs\tensorFLow-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
F:\python\Anaconda3\envs\tensorFLow-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
F:\python\Anaconda3\envs\tensorFLow

In [2]:
# 1 建立模型
model = Sequential()

In [3]:
model.add(Embedding(output_dim=32, input_length=380, input_dim=3800))
model.add(Dropout(0.2))

In [4]:
# 加入LSTM层
model.add(LSTM(32))

In [5]:
model.add(Dense(units=256, activation='relu'))
model.add(Dropout(0.2))

In [6]:
model.add(Dense(units=1, activation='sigmoid'))

In [7]:
# 查看模型的摘要
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 380, 32)           121600    
_________________________________________________________________
dropout_1 (Dropout)          (None, 380, 32)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                8320      
_________________________________________________________________
dense_1 (Dense)              (None, 256)               8448      
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
Total params: 138,625
Trainable params: 138,625
Non-trainable params: 0
_________________________________________________________________


![](./lesson14/17.jpg)

In [8]:
model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer='adam')

In [14]:
# 3. 建立Token（字典）
token = Tokenizer(num_words=3800)
token.fit_on_texts(train_text)
# 4. 将“影评文字”转为“数字列表”
x_train_seq = token.texts_to_sequences(train_text)
x_test_seq = token.texts_to_sequences(test_text)
# 5. 截长补短，让所有“数字列表”长度为380
x_train = sequence.pad_sequences(x_train_seq, maxlen=380)
x_test = sequence.pad_sequences(x_test_seq, maxlen=380)

In [15]:
model.fit(x_train, y_train, batch_size=100, epochs=10, validation_split=0.2, verbose=2)

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
 - 97s - loss: 0.5294 - acc: 0.7286 - val_loss: 0.5153 - val_acc: 0.7364
Epoch 2/10
 - 89s - loss: 0.3149 - acc: 0.8672 - val_loss: 0.5632 - val_acc: 0.7198
Epoch 3/10
 - 82s - loss: 0.2572 - acc: 0.8976 - val_loss: 0.5253 - val_acc: 0.7758
Epoch 4/10
 - 74s - loss: 0.2224 - acc: 0.9132 - val_loss: 0.5590 - val_acc: 0.7694
Epoch 5/10
 - 63s - loss: 0.1940 - acc: 0.9265 - val_loss: 0.5160 - val_acc: 0.7976
Epoch 6/10
 - 67s - loss: 0.1831 - acc: 0.9314 - val_loss: 0.4003 - val_acc: 0.8398
Epoch 7/10
 - 64s - loss: 0.1617 - acc: 0.9393 - val_loss: 0.5410 - val_acc: 0.7880
Epoch 8/10
 - 95s - loss: 0.1408 - acc: 0.9500 - val_loss: 0.4715 - val_acc: 0.8330
Epoch 9/10
 - 104s - loss: 0.1263 - acc: 0.9542 - val_loss: 0.6245 - val_acc: 0.8028
Epoch 10/10
 - 63s - loss: 0.1115 - acc: 0.9606 - val_loss: 0.4448 - val_acc: 0.8478


In [16]:
scores = model.evaluate(x_test, y_test, verbose=1)
scores[1]

25000/25000 [==============================] - 30s 1ms/step


0.85452

### 使用LSTM模型准确率提升至 0.85